In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import zscore
from scipy.stats.mstats import winsorize
from sqlalchemy import create_engine
import warnings

from scipy.stats import jarque_bera
from scipy.stats import normaltest
from scipy.stats import boxcox
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import power_transform
from sklearn.preprocessing import quantile_transform


warnings.filterwarnings('ignore')
sns.set(style="whitegrid")


In [2]:
data_amazon = 'https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/sentiment_labelled_sentences/amazon_cells_labelled.txt'''
amazon_raw = pd.read_csv(data_amazon, delimiter='\t', header=None)
amazon_raw.columns = ['sentence', 'sentiment']
display(amazon_raw.head(20))
amazon_raw.dtypes

,sentence,sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1
5,I have to jiggle the plug to get it to line up...,0
6,If you have several dozen or several hundred c...,0
7,If you are Razr owner...you must have this!,1
8,"Needless to say, I wasted my money.",0
9,What a waste of money and time!.,0


sentence     object
sentiment     int64
dtype: object

In [42]:
def get_average_sentence_length(text):
    new_text = text.replace("!", ".")
    new_text = text.replace("?", ".")
    sentences = new_text.split(".")
    sentences_in_text = []
    for word in sentences:
        sentences_in_text.append(word.count(" ") + 1)
    average_length = sum(sentences_in_text) / len(sentences_in_text)
    return average_length



amazon_raw['sentence_length'] = amazon_raw['sentence'].apply(lambda x: get_average_sentence_length(x))
amazon_raw.groupby('sentiment').sentence_length.median()
amazon_raw['bad_length'] = (amazon_raw.sentence_length > 5) & (amazon_raw.sentence_length < 10)

In [45]:
keywords_bad = ['bad', 'terrible', 'hated', 'worst', 'awful', 'fooled', 'not good', 'waste', 'do not' ]
keywords_good = ['great', 'good', 'fantastic', 'wonderful', 'happy']

for key in keywords_bad:
    amazon_raw[str(key)] = amazon_raw.sentence.str.contains(
        ' ' + str(key) + ' ',
        case=False
)
    
#for word in amazon_raw.sentence:
 #   upper_case = []
  #  if word == amazon_raw.sentence.str.isupper(): #help with this
   #     upper_case.append(word)
    
amazon_raw['allcaps'] = amazon_raw.sentence.str.isupper()
amazon_raw['sentiment_bad'] = (amazon_raw['sentiment'] == 0)
amazon_raw['sentiment_good'] = (amazon_raw['sentiment'] == 1)

data = amazon_raw[keywords_bad + ['allcaps', 'sentence_length', 'bad_length']]
target = amazon_raw['sentiment_bad']

from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
y_pred = bnb.fit(data, target).predict(data)  

In [46]:
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

Number of mislabeled points out of a total 1000 points : 448
